In [7]:
import sys
from datetime import datetime

import gymnasium as gym
import numpy as np
import wandb
from stable_baselines3 import *
from stable_baselines3.common.callbacks import CallbackList
from stable_baselines3.common.logger import HumanOutputFormat
from stable_baselines3.common.logger import Logger as SB3Logger

import sinergym
from sinergym.utils.callbacks import *
from sinergym.utils.constants import *
from sinergym.utils.logger import WandBOutputFormat
from sinergym.utils.rewards import *
from sinergym.utils.wrappers import *

In [8]:
from stable_baselines3.common.env_util import make_vec_env

In [9]:
# Environment ID
environment = 'Eplus-office-mixed-continuous-stochastic-v1'

# Training episodes
episodes = 50

# Name of the experiment
experiment_date = datetime.today().strftime('%Y-%m-%d_%H:%M')
experiment_name = 'EXP_SB3_PPO-' + environment + \
    '-episodes-' + str(episodes)
experiment_name += '_' + experiment_date

In [10]:
extra_conf = {
    'timesteps_per_hour': 6,
    'runperiod': (1, 7, 2024, 31, 7, 2024),
}

In [11]:
env = gym.make(environment,
               env_name=experiment_name,
               weather_files=["JPN_Nagoya.476350_IWEC.epw"],
               reward=ExpReward,
               reward_kwargs = {
                   "energy_weight": 0.5,
                   'temperature_variables': ['perimeter_bot_zn_1_air_temperature',
                     'perimeter_bot_zn_2_air_temperature',
                     'perimeter_bot_zn_3_air_temperature',
                     'perimeter_bot_zn_4_air_temperature',
                     'perimeter_mid_zn_1_air_temperature',
                     'perimeter_mid_zn_2_air_temperature',
                     'perimeter_mid_zn_3_air_temperature',
                     'perimeter_mid_zn_4_air_temperature',
                     'perimeter_top_zn_1_air_temperature',
                     'perimeter_top_zn_2_air_temperature',
                     'perimeter_top_zn_3_air_temperature',
                     'perimeter_top_zn_4_air_temperature',
                                            ],
                   'energy_variables': ['HVAC_electricity_demand_rate'],
                   "range_comfort_winter": (20.0, 23.5),
                   "range_comfort_summer": (24.0, 27.0),
                   "lambda_energy": 0.0001,
               },
              building_config=extra_conf,)

eval_env = gym.make(environment,
                    env_name=experiment_name+'_EVALUATION',
                    weather_files=["JPN_Nagoya.476350_IWEC.epw"],
                    reward=ExpReward,
                    reward_kwargs = {
                        "energy_weight": 0.5,
                        'temperature_variables': ['perimeter_bot_zn_1_air_temperature',
                         'perimeter_bot_zn_2_air_temperature',
                         'perimeter_bot_zn_3_air_temperature',
                         'perimeter_bot_zn_4_air_temperature',
                         'perimeter_mid_zn_1_air_temperature',
                         'perimeter_mid_zn_2_air_temperature',
                         'perimeter_mid_zn_3_air_temperature',
                         'perimeter_mid_zn_4_air_temperature',
                         'perimeter_top_zn_1_air_temperature',
                         'perimeter_top_zn_2_air_temperature',
                         'perimeter_top_zn_3_air_temperature',
                         'perimeter_top_zn_4_air_temperature',
                                                 ],
                        'energy_variables': ['HVAC_electricity_demand_rate'],
                        "range_comfort_winter": (20.0, 23.5),
                        "range_comfort_summer": (24.0, 27.0),
                        "lambda_energy": 0.0001,
                    },
                   building_config=extra_conf,)

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment.
[ENVIRONMENT] (INFO) : Name: EXP_SB3_PPO-Eplus-office-mixed-continuous-stochastic-v1-episodes-50_2025-08-12_15:52
#==============================================================================================#
[MODEL] (INFO) : Working directory created: /workspace/EXP_SB3_PPO-Eplus-office-mixed-continuous-stochastic-v1-episodes-50_2025-08-12_15:52-res1
[MODEL] (INFO) : Model Config is correct.
[MODEL] (INFO) : Building model Output:Variable updated with defined variable names.
[MODEL] (INFO) : Updated building model Output:Meter with meter names.
[MODEL] (INFO) : Building configuration: runperiod updated to {'start_day': 1, 'start_month': 7, 'start_year': 2024, 'end_day': 31, 'end_month': 7, 'end_year': 2024, 'start_weekday': 6, 'n_steps_per_hour': 6}
[MODEL] (INFO) : Updated episode length (seconds): 2678400.0
[MODEL] (INFO) : Updated

In [12]:
env = NormalizeObservation(env)
env = NormalizeAction(env)
env = LoggerWrapper(env)
env = CSVLogger(env)

# run = wandb.init(entity='ojun-info-alpha',
#            project='final',
#            name='first',
#            settings=wandb.Settings(init_timeout=120.0))
# Discomment the following line to log to WandB (remember to set the API key as an environment variable)
# env = WandBLogger(env,
#                  entity='ojun-info-alpha',
#                  project_name='final',
#                  run_name='first',
#                  save_code = True,
#                  dump_frequency = 1000,
#                  artifact_save = False)

eval_env = NormalizeObservation(eval_env)
eval_env = NormalizeAction(eval_env)
eval_env = LoggerWrapper(eval_env)
eval_env = CSVLogger(eval_env)

# In this case, all the hyperparameters are the default ones
model = PPO(
    'MlpPolicy',
    env,
    verbose=1,
    device='cpu',
)

callbacks = []

# Set up Evaluation logging and saving best model
eval_callback = LoggerEvalCallback(
    eval_env=eval_env,
    train_env=env,
    n_eval_episodes=5,
    eval_freq_episodes=10,
    deterministic=True)

callbacks.append(eval_callback)
callback = CallbackList(callbacks)

timesteps = episodes * (env.get_wrapper_attr('timestep_per_episode') - 1)

# wandb logger and setting in SB3
if is_wrapped(env, WandBLogger):
    logger = SB3Logger(
        folder=None,
        output_formats=[
            HumanOutputFormat(
                sys.stdout,
                max_length=200),
            WandBOutputFormat()])
    model.set_logger(logger)

model.learn(
    total_timesteps=timesteps,
    callback=callback,
    log_interval=100)

[WRAPPER NormalizeObservation] (INFO) : Wrapper initialized.
[WRAPPER NormalizeAction] (INFO) : New normalized action space: Box(-1.0, 1.0, (2,), float32)
[WRAPPER NormalizeAction] (INFO) : Wrapper initialized.
[WRAPPER LoggerWrapper] (INFO) : Wrapper initialized.
[WRAPPER CSVLogger] (INFO) : Wrapper initialized.
[WRAPPER NormalizeObservation] (INFO) : Wrapper initialized.
[WRAPPER NormalizeAction] (INFO) : New normalized action space: Box(-1.0, 1.0, (2,), float32)
[WRAPPER NormalizeAction] (INFO) : Wrapper initialized.
[WRAPPER LoggerWrapper] (INFO) : Wrapper initialized.
[WRAPPER CSVLogger] (INFO) : Wrapper initialized.
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
#----------------------------------------------------------------------------------------------#
[ENVIRONMENT] (INFO) : Starting a new episode.
[ENVIRONMENT] (INFO) : Episode 1: EXP_SB3_PPO-Eplus-office-mixed-continuous-stochastic-v1-episodes-50_2025-08-12_15:52
#------------

In [ ]:
model.save(env.get_wrapper_attr('workspace_path') + '/model')

In [ ]:
env.close()
eval_env.close()
#env.wandb_run.finish()

Simulation Progress [Episode 35]: 100%|██████████| 100/100 [00:21<00:00, 25.83%/s, 100% completed]